# score_cells

This notebook is intended to handle scoring cells based on their expression of trait-associated genes. Highly-scored cells are considered disease relevant, whereas lowly-scored cells are considered disease irrelevant.

## Load required packages

In [1]:
import scdrs

from src.utils import make_work_dir, setup
from src.score_cells.utils import load_gene_name_map, munge_magma

## Setup the output and working directories

In [2]:
output_dir, tmp_dir = setup("src/score_cells/output")
work_dir = make_work_dir(tmp_dir)

directory score_cells/output already exists
directory score_cells/tmp already exists
making directory tmp/dd8e26ce30eeeb54ecda009c25c9c027


## Munge the output from MAGMA

scDRS expects a gene set to be used for each trait of interest. Since Rheumatoid Arthritis (RA) is the trait of interest, a gene set should be constructed for RA using the following format:

```
TRAIT   GENESET
RA      [COMMA-DELIMITED LIST OF GENES IN THE RA GENE SET]
```

The RA gene set is constructed by taking the top **n** genes which MAGMA associates with RA using the `gwas_processing` module.

### Change gene IDs to HGNC gene names

In [3]:
gene_ids_to_names = "data/gene_locations/NCBI37.3.gene.loc"
gene_names_header = False
id_column = 0
name_column = 5


gene_name_map = load_gene_name_map(
    gene_name_map=gene_ids_to_names,
    header=gene_names_header,
    id_column=id_column,
    name_column=name_column,
)

### Munge the MAGMA results into the previously described gene set structure for scDRS

In [4]:
ra_associated_genes = "src/gwas_processing/output/gene_analysis.genes.out"
trait = "RA"
num_genes = 1000
delimiter = "\s+"
ra_associated_genes_header = True


ra_gene_set = munge_magma(
    ra_associated_genes,
    trait=trait,
    num_genes=num_genes,
    work_dir=work_dir,
    delimiter=delimiter,
    header=ra_associated_genes_header,
    gene_name_map=gene_name_map,
)

# Load the Seurat data into an AnnData object

Since the AnnData object was created from a Seurat object, the behavior of the AnnData object is analogous to the behavior of a Seurat object in R. Different features of the single-cell data set are accessed using specific attributes (e.g. the `nCount_RNA` attribute) of the AnnData object.

In [5]:
h5ad_file = "src/seurat/output/ra_sc_rna_seq.h5ad"

ann_data = scdrs.util.load_h5ad(h5ad_file, flag_filter_data=False, flag_raw_count=False)

# Load the munged MAGMA output

In [6]:
df_gs = scdrs.util.load_gs(
    ra_gene_set,
    src_species="human",
    dst_species="human",
    to_intersect=ann_data.var_names,
)

# Compute the cell-wise scores

Since no covariates were provided in the 10x single-cell data set, there are no covariates to regress out. However, the `scdrs.preprocess` step in the standard analysis workflow still needs to be run to create a `SCDRS_PARAM` attribute in the AnnData object

In [7]:
scdrs.preprocess(ann_data)

gene_list = df_gs["RA"][0]
gene_weights = df_gs["RA"][1]

In [8]:
# n_control are the number of control groups that scDRS will generate using a Monte Carlo sampling method
# use a range of scores and compare
n_controls = [20, 50, 100, 500, 1000]
results = dict()

for num_controls in n_controls:
    cell_scores = scdrs.score_cell(
        ann_data,
        gene_list,
        gene_weight=gene_weights,
        n_ctrl=num_controls
    )

    results[num_controls] = cell_scores

Computing control scores: 100%|██████████| 1000/1000 [00:55<00:00, 17.93it/s]


## Write the scored cell tables to file

In [9]:
for num_controls in results.keys():
    with open(f"{output_dir}/cell_scores_{num_controls}.tsv", "w") as outfile:
        results[num_controls].reset_index().rename(columns={"index": "cell_name"}).to_csv(outfile, sep="\t", index=False)